In [1]:
## WeightWatcher helps you choose the best pretrained model for your needs.

## You can use WeightWatcher to compare several pretrained models and choose the one with the lowest Log Norm.
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 8,8

from scipy.ndimage.interpolation import zoom
import VGG
import ResNet
import numpy as np
import os
import gradcamutils
from DenseNet import densenet
from PIL import Image
import weightwatcher as ww

Using TensorFlow backend.
/home/reza/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/reza/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/reza/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/reza/.local/lib/python3.6/site-packages/tensorflow/python/framewo

In [2]:
# use this environment flag to change which GPU to use 
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  # specify which GPU(s) to be used

vggModel = VGG.VGG19((352,320,1),4) #set up model architecture
vggModel.summary()
vggModel.load_weights("/home/reza/DeepKneeExplainer/resources/old_models/balanced JSN/VGG19-JSNnewbalance-front-0.8896.h5") #load weights

denseNetModel = densenet.DenseNetImageNet161(input_shape=(352,320,1),classes=4, weights=None)
denseNetModel.summary()
denseNetModel.load_weights("/home/reza/DeepKneeExplainer/resources/old_models/balanced JSN/DenseNet161-JSNnewbalance-XRfront-0.8965.h5")

resNetModel = ResNet.ResNet34(input_shape=(352,320,1),classes=4)
resNetModel.summary()
resNetModel.load_weights("/home/reza/DeepKneeExplainer/resources/old_models/balanced JSN/ResNet34-JSNnewbalance-front-0.8395.h5")



Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 352, 320, 1)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 352, 320, 2)       20        
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 352, 320, 2)       38        
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 176, 160, 2)       0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 176, 160, 4)       76        
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 176, 160, 4)       148       
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 88, 80, 4)         0    

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 352, 320, 1)  0                                            
__________________________________________________________________________________________________
conv2d_161 (Conv2D)             (None, 176, 160, 64) 3200        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_162 (BatchN (None, 176, 160, 64) 256         conv2d_161[0][0]                 
__________________________________________________________________________________________________
activation_161 (Activation)     (None, 176, 160, 64) 0           ba

In [11]:
vggWatcher = ww.WeightWatcher(model=vggModel)
results = vggWatcher.analyze()

vggWatcher.print_results()
vggWatcher.get_summary()

2020-02-17 23:43:50,492 INFO 

python      version 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
numpy       version 1.18.1
tensforflow version 1.14.0
keras       version 2.3.1
2020-02-17 23:43:50,494 INFO Analyzing model 'vgg19' with 27 layers
2020-02-17 23:43:50,542 INFO ### Printing results ###
2020-02-17 23:43:52,039 INFO Check: min: 0.03041860037033362, max: 2.2829769998646285, avg: 1.5622423018363436
2020-02-17 23:43:52,040 INFO Check compound: min: 0.03041860037033362, max: 1.9394260118351636, avg: 1.311011840157124
2020-02-17 23:43:52,041 INFO CheckTF: min: False, max: True, avg: 0.2835820895522388
2020-02-17 23:43:52,042 INFO CheckTF compound: min: 0.0, max: 1.0, avg: 0.29385964912280704
2020-02-17 23:43:52,042 INFO Norm: min: 10.328792572021484, max: 20.418109893798828, avg: 15.373451232910156
2020-02-17 23:43:52,043 INFO Norm compound: min: 10.328792572021484, max: 20.418109893798828, avg: 15.373451232910156
2020-02-17 23:43:52,044 INFO LogNorm: min: 1.0140495300292969

{'check': 1.5622423018363436,
 'check_compound': 1.311011840157124,
 'checkTF': 0.2835820895522388,
 'checkTF_compound': 0.29385964912280704,
 'norm': 15.373451,
 'norm_compound': 15.373451,
 'lognorm': 1.1620326,
 'lognorm_compound': 1.1620326,
 'normX': 17.161222,
 'normX_compound': 17.161222,
 'lognormX': 1.2056782,
 'lognormX_compound': 1.2056782}

In [12]:
vggWatcher.analyze(layers=ww.LAYER_TYPE.CONV2D)

for layer_id, result in results.items():
    for slice_id, summary in result.items():
        if not str(slice_id).isdigit() or "lognorm" not in summary:
            continue
        lognorm = summary["lognorm"]
        print("Layer {}, Slice {}: Lognorm: {}".format(layer_id, slice_id, lognorm))

2020-02-17 23:43:57,924 INFO Analyzing model 'vgg19' with 27 layers
2020-02-17 23:43:57,956 INFO ### Printing results ###
2020-02-17 23:43:59,392 INFO Check: min: 0.5231152375539144, max: 2.2829769998646285, avg: 1.5963780696821568
2020-02-17 23:43:59,394 INFO Check compound: min: 0.8410260145476288, max: 1.9394260118351636, avg: 1.5433927278298647
2020-02-17 23:43:59,395 INFO CheckTF: min: False, max: True, avg: 0.2900763358778626
2020-02-17 23:43:59,396 INFO CheckTF compound: min: 0.0, max: 1.0, avg: 0.34895833333333337


Layer 23, Slice 0: Lognorm: 1.3100155591964722
Layer 25, Slice 0: Lognorm: 1.0140495300292969


In [13]:
resnetWatcher = ww.WeightWatcher(model=resNetModel)
results = resnetWatcher.analyze()

resnetWatcher.print_results()
resnetWatcher.get_summary()

2020-02-17 23:44:19,611 INFO 

python      version 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
numpy       version 1.18.1
tensforflow version 1.14.0
keras       version 2.3.1
2020-02-17 23:44:19,614 INFO Analyzing model 'model_1' with 127 layers
2020-02-17 23:44:19,869 INFO ### Printing results ###
2020-02-17 23:44:24,032 INFO Check: min: 0.03055666800210663, max: 1.174803615616177, avg: 0.7524212801186786
2020-02-17 23:44:24,033 INFO Check compound: min: 0.03055666800210663, max: 1.171422623526532, avg: 0.6695346588121719
2020-02-17 23:44:24,034 INFO CheckTF: min: False, max: True, avg: 0.8293515358361775
2020-02-17 23:44:24,035 INFO CheckTF compound: min: 0.0, max: 1.0, avg: 0.7297297297297297
2020-02-17 23:44:24,036 INFO Norm: min: 2.481132745742798, max: 18.79686164855957, avg: 8.44815731048584
2020-02-17 23:44:24,037 INFO Norm compound: min: 2.522228717803955, max: 18.74276351928711, avg: 8.113859176635742
2020-02-17 23:44:24,038 INFO LogNorm: min: 0.3946499824523926, max:

{'check': 0.7524212801186786,
 'check_compound': 0.6695346588121719,
 'checkTF': 0.8293515358361775,
 'checkTF_compound': 0.7297297297297297,
 'norm': 8.448157,
 'norm_compound': 8.113859,
 'lognorm': 0.8404216,
 'lognorm_compound': 0.8240068,
 'normX': 7.6573663,
 'normX_compound': 7.1977673,
 'lognormX': 0.70779043,
 'lognormX_compound': 0.68141043}

In [14]:
resnetWatcher.analyze(layers=ww.LAYER_TYPE.CONV2D)

for layer_id, result in results.items():
    for slice_id, summary in result.items():
        if not str(slice_id).isdigit() or "lognorm" not in summary:
            continue
        lognorm = summary["lognorm"]
        print("Layer {}, Slice {}: Lognorm: {}".format(layer_id, slice_id, lognorm))

2020-02-17 23:44:47,209 INFO Analyzing model 'model_1' with 127 layers
2020-02-17 23:44:47,442 INFO ### Printing results ###
2020-02-17 23:44:51,547 INFO Check: min: 0.03055666800210663, max: 1.174803615616177, avg: 0.75461311809406
2020-02-17 23:44:51,548 INFO Check compound: min: 0.03055666800210663, max: 1.171422623526532, avg: 0.6850104940206393
2020-02-17 23:44:51,549 INFO CheckTF: min: False, max: True, avg: 0.8321917808219178
2020-02-17 23:44:51,549 INFO CheckTF compound: min: 0.0, max: 1.0, avg: 0.75
2020-02-17 23:44:51,550 INFO Norm: min: 2.481132745742798, max: 18.79686164855957, avg: 8.44815731048584
2020-02-17 23:44:51,551 INFO Norm compound: min: 2.522228717803955, max: 18.74276351928711, avg: 8.113859176635742
2020-02-17 23:44:51,551 INFO LogNorm: min: 0.3946499824523926, max: 1.2740854024887085, avg: 0.8404216170310974
2020-02-17 23:44:51,552 INFO LogNorm compound: min: 0.4017566442489624, max: 1.2728333473205566, avg: 0.8240067958831787
2020-02-17 23:44:51,553 INFO Norm

Layer 5, Slice 0: Lognorm: 0.45505785942077637
Layer 5, Slice 1: Lognorm: 0.466656893491745
Layer 5, Slice 2: Lognorm: 0.4717792868614197
Layer 5, Slice 3: Lognorm: 0.46813076734542847
Layer 5, Slice 4: Lognorm: 0.46588632464408875
Layer 5, Slice 5: Lognorm: 0.47582072019577026
Layer 5, Slice 6: Lognorm: 0.464699387550354
Layer 5, Slice 7: Lognorm: 0.4623936414718628
Layer 5, Slice 8: Lognorm: 0.46986302733421326
Layer 7, Slice 0: Lognorm: 0.457430899143219
Layer 10, Slice 0: Lognorm: 0.4125364422798157
Layer 10, Slice 1: Lognorm: 0.4140439033508301
Layer 10, Slice 2: Lognorm: 0.4220924377441406
Layer 10, Slice 3: Lognorm: 0.4108112156391144
Layer 10, Slice 4: Lognorm: 0.4176965057849884
Layer 10, Slice 5: Lognorm: 0.4095495045185089
Layer 10, Slice 6: Lognorm: 0.41007551550865173
Layer 10, Slice 7: Lognorm: 0.4107871353626251
Layer 10, Slice 8: Lognorm: 0.41786688566207886
Layer 14, Slice 0: Lognorm: 0.42499804496765137
Layer 14, Slice 1: Lognorm: 0.40750980377197266
Layer 14, Slice 2

In [15]:
densenetWatcher = ww.WeightWatcher(model=denseNetModel)
results = densenetWatcher.analyze()

densenetWatcher.print_results()
densenetWatcher.get_summary()

2020-02-17 23:45:34,546 INFO 

python      version 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
numpy       version 1.18.1
tensforflow version 1.14.0
keras       version 2.3.1
2020-02-17 23:45:34,550 INFO Analyzing model 'densenet' with 567 layers
2020-02-17 23:45:34,711 INFO ### Printing results ###
2020-02-17 23:45:47,756 INFO Check: min: 0.06097311385698125, max: 2.5768312503419826, avg: 1.0410752223320638
2020-02-17 23:45:47,757 INFO Check compound: min: 0.06097311385698125, max: 1.9540489048209366, avg: 0.618539129806069
2020-02-17 23:45:47,758 INFO CheckTF: min: False, max: True, avg: 0.8863636363636364
2020-02-17 23:45:47,758 INFO CheckTF compound: min: 0.0, max: 1.0, avg: 0.484472049689441
2020-02-17 23:45:47,759 INFO Norm: min: 9.715526580810547, max: 16.15900421142578, avg: 12.937265396118164
2020-02-17 23:45:47,760 INFO Norm compound: min: 9.715526580810547, max: 16.15900421142578, avg: 12.937265396118164
2020-02-17 23:45:47,761 INFO LogNorm: min: 0.9874663352966309, 

{'check': 1.0410752223320638,
 'check_compound': 0.618539129806069,
 'checkTF': 0.8863636363636364,
 'checkTF_compound': 0.484472049689441,
 'norm': 12.937265,
 'norm_compound': 12.937265,
 'lognorm': 1.0979404,
 'lognorm_compound': 1.0979404,
 'normX': 15.923746,
 'normX_compound': 15.923746,
 'lognormX': 1.1887772,
 'lognormX_compound': 1.1887772}

In [16]:
densenetWatcher.analyze(layers=ww.LAYER_TYPE.CONV2D)

for layer_id, result in results.items():
    for slice_id, summary in result.items():
        if not str(slice_id).isdigit() or "lognorm" not in summary:
            continue
        lognorm = summary["lognorm"]
        print("Layer {}, Slice {}: Lognorm: {}".format(layer_id, slice_id, lognorm))

2020-02-17 23:47:03,220 INFO Analyzing model 'densenet' with 567 layers
2020-02-17 23:47:03,397 INFO ### Printing results ###
2020-02-17 23:47:16,470 INFO Check: min: 0.06097311385698125, max: 2.5768312503419826, avg: 1.0422549530044871
2020-02-17 23:47:16,471 INFO Check compound: min: 0.06097311385698125, max: 1.9540489048209366, avg: 0.6217305727395739
2020-02-17 23:47:16,472 INFO CheckTF: min: False, max: True, avg: 0.8874841972187105
2020-02-17 23:47:16,473 INFO CheckTF compound: min: 0.0, max: 1.0, avg: 0.4875
2020-02-17 23:47:16,474 INFO Norm: min: 9.715526580810547, max: 16.15900421142578, avg: 12.937265396118164
2020-02-17 23:47:16,475 INFO Norm compound: min: 9.715526580810547, max: 16.15900421142578, avg: 12.937265396118164
2020-02-17 23:47:16,476 INFO LogNorm: min: 0.9874663352966309, max: 1.2084145545959473, avg: 1.097940444946289
2020-02-17 23:47:16,477 INFO LogNorm compound: min: 0.9874663352966309, max: 1.2084145545959473, avg: 1.097940444946289
2020-02-17 23:47:16,478 I

Layer 137, Slice 0: Lognorm: 0.9874663352966309
Layer 393, Slice 0: Lognorm: 1.2084145545959473


In [9]:
## WeightWatcher helps you choose the best pretrained model for your needs.

## You can use WeightWatcher to compare several pretrained models and choose the one with the lowest Log Norm.